In [35]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser, StrOutputParser

In [2]:
list_instructions = CommaSeparatedListOutputParser().get_format_instructions()
list_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [9]:
chat_template = ChatPromptTemplate([("human", "I wanted a {flower_name} to decorate my room \n"+list_instructions)])
chat_template.messages[0].prompt.template

'I wanted a {flower_name} to decorate my room \nYour response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [10]:
chat = ChatOpenAI(model="gpt-4",
                  seed=365,
                  temperature = 0,
                  max_tokens = 100)

In [11]:
list_parser = CommaSeparatedListOutputParser()

In [14]:
chat_template_result = chat_template.invoke({"flower_name":"rose"})

In [15]:
chat_result = chat.invoke(chat_template_result)

In [16]:
list_parser.invoke(chat_result)

['rose', 'vase', 'water', 'rose food', 'pruning shears', 'gloves']

using langchain expression language

piped components to form chains

In [ ]:
'''
a. Uses a pipe symbol to connect components
b. Connecting them means that the output of one will serve as the input to the next.
c. We can apply the invoke() method to each component.'''
chain = chat_template | chat | list_parser

In [18]:
chain.invoke({"flower_name":"lili"})

['lili plant',
 'lili flower vase',
 'lili wall art',
 'lili scented candle',
 'lili patterned curtains',
 'lili themed bedding',
 'lili throw pillows',
 'lili rug',
 'lili wall clock',
 'lili picture frame',
 'lili lampshade']

Batching

In [20]:
chat_template1 = ChatPromptTemplate.from_messages([
    ("human", "I have recently adopted a {pet} which is a {breed}. Could you suggest several training tips?")
])

In [21]:
chain = chat_template1 | chat

In [22]:
chain.invoke({"pet":"dog", "breed":"doberman"})

AIMessage(content="1. Start Training Early: Dobermans are intelligent and learn quickly, so it's best to start training them as soon as possible. \n\n2. Use Positive Reinforcement: Always reward your Doberman for good behavior. This can be in the form of treats, toys, praise, or extra playtime. \n\n3. Be Consistent: Consistency is key when training any dog. Make sure to use the same commands and signals each time, and ensure all family members are doing the same", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 26, 'total_tokens': 126, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-C8PT4oPXYhBc3uClqjFtEN01bL5hn', 'service_tier': 'default', 'finish_reason': 'length', 'logprobs': None}, id='run-

In [23]:
# the invoke() doesnt allow for feeding several inputs at once.
# we need to write invoke() for no of input times
# instead of this we can use batch() to runs invoke() parallel.
# it takes less time as comparing to running several subsequant invoke() method.
chain.batch([{"pet":"cat", "breed":"Persian"},{"pet":"sheep", "breed":"Valais Blacknose"}])

[AIMessage(content='1. Start Early: The earlier you start training your Persian cat, the better. They are more likely to adapt to new behaviors when they are young.\n\n2. Use Positive Reinforcement: Always use positive reinforcement when training your Persian cat. This could be in the form of treats, praises, or petting. Never punish your cat as it can lead to fear and aggression.\n\n3. Litter Training: Persian cats are usually good with litter boxes. Make sure the litter box is easily accessible and', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 25, 'total_tokens': 125, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-C8PYOaxHzi5OAFiI4wPKHrmvgR5XJ', 'service_tier': 'default', 'finish_reason'

Streaming

In [25]:
# This will return generator object
response = chain.stream({"pet":"cat", "breed":"Persian"})
response

<generator object RunnableSequence.stream at 0x7f8c6fb8a490>

In [28]:
for i in response:
    print(i.content, end="")

1. Start Early: The earlier you start training your Persian cat, the better. They are more likely to adapt to new behaviors when they are young.

2. Use Positive Reinforcement: Always reward your cat for good behavior. This could be in the form of treats, praise, or petting. Never punish your cat as it can lead to fear and aggression.

3. Litter Training: Persian cats are usually good with litter boxes. Make sure the litter box is easily accessible and clean it

Runnable and Runnable Sequence classes

Runnable => a unit of work that can be invoked, batched, streamed, transformed and composed.

In [30]:
# sequence of runnable, where the output of each is the input of the next
type(chain)

langchain_core.runnables.base.RunnableSequence

piping chains and the RunnablePassthrough Class

In [ ]:
from langchain_core.runnables import RunnablePassthrough #langchain identity function

In [33]:
RunnablePassthrough().invoke("hi")

'hi'

pipeing chain together

In [37]:
chat_template_tools = ChatPromptTemplate.from_template(" what are the five most important tools a {job title} needs? Answer only by listing the tools.")

chat_template_strategy = ChatPromptTemplate.from_template("Considering the tools provided, develop a strategy for effectivly learning and mastering them: {tools}")

In [38]:
str_parser = StrOutputParser()

In [49]:
chain_tools = chat_template_tools | chat | str_parser
chain_strategy = chat_template_strategy | chat | str_parser

In [52]:
print(chain_tools.invoke({"job title": "data scientist"}))

1. Python
2. R Programming Language
3. SQL (Structured Query Language)
4. Tableau
5. Apache Hadoop


In [53]:
chain_strategy.invoke({"tools": """1. Python
2. R Programming Language
3. SQL (Structured Query Language)
4. Tableau
5. Apache Hadoop"""})

"1. Python: Start with the basics of Python, such as variables, data types, loops, and functions. Use online resources like Codecademy, Coursera, or edX for structured learning. Practice coding problems on platforms like LeetCode or HackerRank. Work on small projects, like a simple game or a web scraper, to apply what you've learned. As you get more comfortable, delve into more complex topics like object-oriented programming and libraries for data analysis like pandas and NumPy"

make 2 chain to single chain using runnable passthrough

flow:
{job title: scientist} -> chat_template_tools -> chat_prompt_value object -> chat -> AIMessage -> string parser -> result -> runnablepassthrough -> tools: as a output

In [54]:
chain_tools = chat_template_tools | chat | str_parser | {"tools":RunnablePassthrough()}

In [55]:
print(chain_tools.invoke({"job title": "data scientist"}))

{'tools': '1. Python\n2. R Programming Language\n3. SQL (Structured Query Language)\n4. Tableau\n5. Hadoop'}


In [56]:
chain_combined = chain_tools | chain_strategy

In [57]:
print(chain_combined.invoke({"job title": "data scientist"}))

1. Python: Start with the basics of Python, such as variables, data types, operators, control flow, and loops. Then, move on to more advanced topics like functions, classes, and exceptions. Use online resources like Codecademy, Coursera, or edX for structured learning. Practice coding problems on platforms like LeetCode or HackerRank. Work on small projects to apply what you've learned.

2. R Programming Language: Begin with the basics of R, such as data


Graphing runnables using grandalf

In [58]:
chain_combined.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
  +--------------------+   
  | ChatPromptTemplate |   
  +--------------------+   
            *              
            *              
            *              
      +------------+       
      | ChatOpenAI |       
      +------------+       
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  
            *              
            *              
            *              
     +-------------+       
     | Passthrough |       
     +-------------+       
            *              
            *              
            *       

Introduction to RunnableParallelClass

In [59]:
from langchain_core.runnables import RunnableParallel

In [60]:
chat_template_books = ChatPromptTemplate.from_template(
    '''
    Suggest three of the best intermediate-level {programming language} books. 
    Answer only by listing the books.
    '''
)

chat_template_projects = ChatPromptTemplate.from_template(
    '''
    Suggest three interesting {programming language} projects suitable for intermediate-level programmers. 
    Answer only by listing the projects.
    '''
)

In [61]:
chain_books = chat_template_books | chat | str_parser

chain_projects = chat_template_projects | chat | str_parser

In [62]:
chain_parallel = RunnableParallel({"books": chain_books, "projects":chain_projects})

In [65]:
chain_parallel.invoke({"programming language": "python"})

{'books': '1. "Fluent Python: Clear, Concise, and Effective Programming" by Luciano Ramalho\n2. "Python Cookbook: Recipes for Mastering Python 3" by David Beazley and Brian K. Jones\n3. "Effective Python: 90 Specific Ways to Write Better Python" by Brett Slatkin',
 'projects': '1. Building a Weather Forecast Application using APIs.\n2. Developing a Web Scraper for E-commerce Websites.\n3. Creating a Personal Finance Tracker with Data Visualization.'}

In [66]:
chain_parallel.get_graph().print_ascii()

            +-------------------------------+              
            | Parallel<books,projects>Input |              
            +-------------------------------+              
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatOpenAI |                      | ChatOpenAI |     
    +------------+                      +------------+     
           *                                   *           
           *                            

Piping the RunnableParallel with another Runnables

In [67]:
chat_template_time = ChatPromptTemplate.from_template(
     '''
     I'm an intermediate level programmer.
     
     Consider the following literature:
     {books}
     
     Also, consider the following projects:
     {projects}
     
     Roughly how much time would it take me to complete the literature and the projects?
     
     '''
)

In [69]:
chain_time = (RunnableParallel({'books': chain_books, "projects":chain_projects})
            | chat_template_time | chat | str_parser)

In [71]:
chain_time.invoke({"programming language":"python"})

'The time it takes to complete the literature and the projects can vary greatly depending on several factors such as your current skill level, the amount of time you can dedicate each day, your reading speed, and how quickly you grasp new concepts.\n\nFor the literature:\n\n1. "Fluent Python: Clear, Concise, and Effective Programming" by Luciano Ramalho - This book is around 800 pages. If you read and practice for about 2 hours a day, it might take you around'

In [72]:
chain_time.get_graph().print_ascii()

            +-------------------------------+              
            | Parallel<books,projects>Input |              
            +-------------------------------+              
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatOpenAI |                      | ChatOpenAI |     
    +------------+                      +------------+     
           *                                   *           
           *                            

RunnableLambda

In [ ]:
find_sum = lambda x:sum(x)
find_sum([1,2,3])

6

In [78]:
find_squar= lambda x:x**2
find_squar(24)

576

In [79]:
from langchain_core.runnables import RunnableLambda

In [82]:
runnable_sum = RunnableLambda(lambda x : sum(x))
runnable_sum.invoke([1,2,3])

6

In [83]:
runnable_square = RunnableLambda(lambda x:x**2)
runnable_square.invoke(9)

81

In [84]:
chain = runnable_sum | runnable_square

In [85]:
chain.invoke([1,2,4])

49

In [86]:
chain.get_graph().print_ascii()

+-------------+  
| LambdaInput |  
+-------------+  
        *        
        *        
        *        
   +--------+    
   | Lambda |    
   +--------+    
        *        
        *        
        *        
   +--------+    
   | Lambda |    
   +--------+    
        *        
        *        
        *        
+--------------+ 
| LambdaOutput | 
+--------------+ 


The @chain Decorator

In [87]:
def find_sum(x):
    return sum(x)

def find_square(x):
    return x**2

In [91]:
chain1 = RunnableLambda(find_sum) | RunnableLambda(find_square) 

In [92]:
chain1.invoke([1,2,3])

36

In [94]:
from langchain_core.runnables import chain

In [95]:
@chain
def runnable_sum(x):
    return sum(x)

@chain
def runnable_square(x):
    return x**2

In [96]:
type(runnable_square), type(runnable_sum)

(langchain_core.runnables.base.RunnableLambda,
 langchain_core.runnables.base.RunnableLambda)

In [97]:
chain2 = runnable_sum | runnable_square

In [98]:
chain2.invoke([1,2,3])

36